## 1 - Import

In [8]:
import os
import tiktoken
import difflib
import pandas as pd
import io
import sys
from dotenv import load_dotenv
from firecrawl import FirecrawlApp
from langchain_openai import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings, AzureOpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import requests
import urllib.parse
import fitz
import re
import numpy as np
from langchain_chroma import Chroma

In [9]:
load_dotenv()

True

## 2 - Models

In [10]:
llm_4omini = AzureChatOpenAI(
    azure_deployment="gpt-4o-mini",  # or your deployment
    api_version="2025-01-01-preview",  # or your api version
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    azure_endpoint=os.getenv('Azure_OpenAI_emb_3_large_azure_endpoint'),
    openai_api_key=os.getenv('Azure_OpenAI_emb_3_large_api_key'),
    openai_api_version="2023-05-15",
    chunk_size=100 # pas trop de risque  mais cela va etre long... 
)


In [94]:
emb = embeddings.embed_query("test")
print(emb)

[-0.02472612281120704, -0.008289816854198067, -0.009792432265614928, 0.04675216264354094, 0.006944397518735483, 0.026612483173239047, -0.01257573916172952, 0.0721995389150771, 0.010939044032163796, 0.04249860379573062, -0.01459155615661382, 0.02350553702330018, -0.0025590704954245903, -0.022155493671218447, 0.00846550713398378, 0.007906071903582921, 0.005506509492422184, 0.012927120652623528, -0.026686458125919618, 0.0022123130211497313, 0.012242851874666944, -0.02576177214873505, 0.004265117492374317, 0.019214989767826998, -0.03275240238308137, 0.0210643635848413, 0.03835600644561858, 0.007129334900436914, -0.014674778211210108, -0.015248084560145833, 0.008359167790259489, 0.05366881606256153, 0.0025290181126904464, 0.001485278026647475, 0.0007154763499382553, -0.0013234579340740463, 0.03743131860578885, -0.012335321031178952, 0.023283612165258465, 0.0031716753976875943, 0.009727704414850073, -0.029608469688124802, -0.0023787566646810187, 0.017069716140090403, -0.04005742849462658, -0

## 3 - Data ingestion pipelines

In [12]:
def count_tokens(text: str, model: str = "gpt-4o") -> int:
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

In [13]:
def chunk_stat_token(chunks_aml5):
    chunk_lengths = [count_tokens(chunk) for chunk in chunks_aml5]
    print("Nombre de chunks:", len(chunk_lengths))
    print("Nombre de token des chunks:", chunk_lengths)
    print("Nombre de token moyen par chunk :", np.mean(chunk_lengths))
    print("Nombre de token max par chunk :", np.max(chunk_lengths))
    print("Nombre de token min par chunk :", np.min(chunk_lengths))

In [14]:
def ajouter_espace_articles(liste_textes):

    pattern = r'(Article \d+) ([A-Za-zÀ-ÿ])'
    resultat = []
    
    for item in liste_textes:
        # Si l'élément est une chaîne, appliquer regex
        if isinstance(item, str):
            resultat.append(re.sub(pattern, r'\1  \2', item))
        # Si l'élément est une liste, ignorer
        else:
            resultat.append(item)
    
    return resultat

### 3.1 - Step 1: Input url

In [15]:
url_aml_5="http://publications.europa.eu/resource/celex/32015L0849"
url_crr="http://publications.europa.eu/resource/celex/32013R0575"

In [16]:
url_aml_5_pdf="https://eur-lex.europa.eu/legal-content/FR/TXT/PDF/?uri=CELEX:32015L0849"
url_crr_pdf="https://eur-lex.europa.eu/legal-content/FR/TXT/PDF/?uri=CELEX:32013R0575"
url_dsp2_pdf="https://eur-lex.europa.eu/legal-content/FR/TXT/PDF/?uri=CELEX:32015L2366"

### 3.2 - Step 2: URL to local pdf

In [17]:
def pipe_1_url_to_pdf(url, save_path):
    response = requests.get(url, 
                            headers={"Accept": "application/pdf"})
    # tres important pour ne pas renvoyer rdf ou autre format...
    with open(save_path, "wb") as f:
        f.write(response.content)

In [18]:
pipe_1_url_to_pdf(url_aml_5_pdf,
                  "/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_scrapped/aml5.pdf")

In [19]:
pipe_1_url_to_pdf(url_dsp2_pdf,
                  "/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_scrapped/dsp2.pdf")

In [20]:
pipe_1_url_to_pdf(url_crr_pdf,
                  "/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_scrapped/crr.pdf")

### 3.3 - Step 3: PDf to txt (PyMuPDF)

PyMuPDF is a high-performance Python library for data extraction, analysis, conversion & manipulation of PDF (and other) documents

In [21]:
def pipe_2_extract_text_with_pymupdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text.strip() 

In [22]:
url_aml_pdf_txt_fitz = pipe_2_extract_text_with_pymupdf("/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_scrapped/aml5.pdf")

In [23]:
url_dsp2_pdf_txt_fitz = pipe_2_extract_text_with_pymupdf("/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_scrapped/dsp2.pdf")


In [24]:
url_crr_pdf_txt_fitz = pipe_2_extract_text_with_pymupdf("/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_scrapped/crr.pdf")

In [25]:
count_tokens(url_aml_pdf_txt_fitz)

45232

In [26]:
count_tokens(url_crr_pdf_txt_fitz)

407103

In [27]:

count_tokens(url_dsp2_pdf_txt_fitz)

86906

### 3.4 - Step 4: Txt cleaning

In [28]:
def pipe_3_nettoyer_texte(texte):
    # 1. Supprime les caractères de coupure de mot suivis de retour à la ligne
    texte = re.sub(r'\xad\n', '', texte)

    # 2. \ndénommé -> dénommé, \nEn -> En
    texte = re.sub(r'\n(?=\w)', '', texte)

    # 3. Supprime les retours à la ligne devant une parenthèse contenant des chiffres (\n(2) -> (2))
    texte = re.sub(r'\n(?=\(\d+\))', '', texte)

    # 4. Remplace les apostrophes échappées (l\'Union -> l'Union)
    texte = re.sub(r"\\'", "'", texte)

    # 5. Supprime tout autre backslash isolé non nécessaire
    texte = re.sub(r'\\', '', texte)

    return texte

In [29]:
crr_pdf_txt_fitz_clean= pipe_3_nettoyer_texte(url_crr_pdf_txt_fitz)
aml_pdf_txt_fitz_clean= pipe_3_nettoyer_texte(url_aml_pdf_txt_fitz)
dsp2_pdf_txt_fitz_clean= pipe_3_nettoyer_texte(url_dsp2_pdf_txt_fitz)


In [30]:
print("Règlement UE AML nombre de tokens après nettoyage: ", count_tokens(crr_pdf_txt_fitz_clean))
print("Règlement UE DSP2 nombre de tokens après nettoyage:", count_tokens(aml_pdf_txt_fitz_clean))
print("Règlement UE CRR nombre de tokens après nettoyage: ", count_tokens(dsp2_pdf_txt_fitz_clean))

Règlement UE AML nombre de tokens après nettoyage:  360213
Règlement UE DSP2 nombre de tokens après nettoyage: 42527
Règlement UE CRR nombre de tokens après nettoyage:  81086


## 3 - Chunking strategies

- crr_pdf_txt_fitz_clean
- aml_pdf_txt_fitz_clean
- dsp2_pdf_txt_fitz_clean

### 3.1  - Statégie 1: by Article

In [31]:
def chunker_1_v1_step_1(input_data_to_chunk: str ) -> list:
    """Optimal chunker by article tested and approved on AML5 + CRR

    Args:
        input_data_to_chunk (str): texte entrée a chunker apres conversion de pdf

    Returns:
        list: of chunks
    """
    article_splitter = RecursiveCharacterTextSplitter(
    separators=   ["Article"],
    chunk_size= 100, 
    chunk_overlap=0)
    chunks_list= article_splitter.split_text(input_data_to_chunk)
    return chunks_list

In [32]:
article_splitter = RecursiveCharacterTextSplitter(
    separators=   ["Article"],
    chunk_size= 100, 
    chunk_overlap=0)

#### 3.1.0  - Application a DSP2

In [33]:
def chunker_1_v1_step_2(beginning: int,end:int,chunks_dsp2_list:list) -> dict:
    """Step2: select beginning and end of a chunk. Add a space and transform to dict

    Args:
        beginning (int): _description_
        end (int): _description_
        chunks_dsp2_list (list): _description_

    Returns:
        _type_: _description_
    """
    chunks_dsp2_list_1 = chunks_dsp2_list[beginning:end]
    def ajouter_espace_article(texte): return re.sub(r'(Article\s+\d+)\s*([^\d\s])', r'\1  \2', texte)
    chunks_dsp2_list_2 = [ajouter_espace_article(i) for i in chunks_dsp2_list_1]
    chunks_dsp2_dict = {i[:11].strip(): i[11:].strip() for i in chunks_dsp2_list_2}
    return chunks_dsp2_dict

In [34]:
chunks_dsp2_v1_1= chunker_1_v1_step_1(dsp2_pdf_txt_fitz_clean)
chunks_dsp2_v1_2= chunker_1_v1_step_2(1,118,chunks_dsp2_v1_1)

#### 3.1.1  - Application AML 5

In [35]:
chunks_aml5 = article_splitter.split_text(crr_pdf_txt_fitz_clean)
chunks_aml5 = chunks_aml5[1:70] #attention écarte premier 
chunk_stat_token(chunks_aml5)
chunks_aml5

Nombre de chunks: 69
Nombre de token des chunks: [224, 42, 62, 9747, 179, 414, 625, 920, 348, 413, 796, 99, 428, 350, 789, 116, 276, 902, 398, 1650, 876, 123, 144, 146, 126, 901, 465, 1636, 706, 137, 401, 251, 518, 69, 224, 1173, 154, 429, 448, 106, 324, 386, 187, 191, 240, 910, 204, 793, 1719, 166, 116, 1359, 267, 949, 195, 435, 379, 158, 235, 962, 128, 299, 1061, 161, 175, 288, 364, 169, 231]
Nombre de token moyen par chunk : 592.2028985507246
Nombre de token max par chunk : 9747
Nombre de token min par chunk : 42


["Article premier Champ d'application Le présent règlement fixe des règles uniformes concernant les exigences prudentielles générales que tous les établissements faisant l'objet d'une surveillance en vertu de la directive 2013/36/UE respectent en ce qui concerne: a) les exigences de fonds propres relatives aux éléments entièrement quantifiables, uniformes et normalisés de risque de crédit, de risque de marché, de risque opérationnel et de risque de règlement; b) les exigences limitant les grands risques; c) après l'entrée en vigueur de l'acte délégué visé à l'article 460, les exigences de liquidité relatives aux éléments entièrement quantifiables, uniformes et normalisés de risque de liquidité; d) les obligations de déclaration en ce qui concerne les points a), b) et c) et le levier; e) les obligations de publication. Le présent règlement ne régit pas les exigences de publication applicables aux autorités compétentes dans le domaine de la régulation et de la surveillance prudentielles 

In [36]:
# exemple 1: AML 5
chunks_aml5_dic = {i[:10] : i[10:].lstrip() for i in chunks_aml5}
chunks_aml5_dic

{'Article pr': "emier Champ d'application Le présent règlement fixe des règles uniformes concernant les exigences prudentielles générales que tous les établissements faisant l'objet d'une surveillance en vertu de la directive 2013/36/UE respectent en ce qui concerne: a) les exigences de fonds propres relatives aux éléments entièrement quantifiables, uniformes et normalisés de risque de crédit, de risque de marché, de risque opérationnel et de risque de règlement; b) les exigences limitant les grands risques; c) après l'entrée en vigueur de l'acte délégué visé à l'article 460, les exigences de liquidité relatives aux éléments entièrement quantifiables, uniformes et normalisés de risque de liquidité; d) les obligations de déclaration en ce qui concerne les points a), b) et c) et le levier; e) les obligations de publication. Le présent règlement ne régit pas les exigences de publication applicables aux autorités compétentes dans le domaine de la régulation et de la surveillance prudentiel

#### 3.1.2  - Application CRR

In [37]:
chunks_crr = article_splitter.split_text(crr_pdf_txt_fitz_clean)
chunks_crr = chunks_crr[1:]
chunk_stat_token(chunks_crr)
chunks_crr

Nombre de chunks: 1373
Nombre de token des chunks: [224, 42, 62, 9747, 179, 414, 625, 920, 348, 413, 796, 99, 428, 350, 789, 116, 276, 902, 398, 1650, 876, 123, 144, 146, 126, 901, 465, 1636, 706, 137, 401, 251, 518, 69, 224, 1173, 154, 429, 448, 106, 324, 386, 187, 191, 240, 910, 204, 793, 1719, 166, 116, 1359, 267, 949, 195, 435, 379, 158, 235, 962, 128, 299, 1061, 161, 175, 288, 364, 169, 231, 932, 62, 46, 696, 126, 152, 607, 136, 860, 329, 462, 275, 272, 599, 1183, 688, 129, 632, 133, 562, 89, 295, 770, 537, 485, 584, 436, 443, 356, 798, 83, 851, 176, 381, 469, 1830, 406, 511, 243, 298, 675, 450, 240, 1301, 746, 701, 656, 361, 117, 499, 571, 358, 181, 292, 1027, 912, 852, 340, 324, 2452, 40, 137, 1081, 289, 838, 206, 1047, 449, 566, 359, 220, 238, 817, 498, 775, 375, 119, 1242, 836, 461, 1248, 936, 1217, 2165, 1531, 1105, 293, 278, 904, 365, 965, 688, 2044, 261, 926, 338, 1398, 87, 118, 151, 994, 331, 630, 558, 341, 491, 799, 348, 1161, 950, 1789, 1177, 966, 1010, 1149, 642, 697, 4

["Article premier Champ d'application Le présent règlement fixe des règles uniformes concernant les exigences prudentielles générales que tous les établissements faisant l'objet d'une surveillance en vertu de la directive 2013/36/UE respectent en ce qui concerne: a) les exigences de fonds propres relatives aux éléments entièrement quantifiables, uniformes et normalisés de risque de crédit, de risque de marché, de risque opérationnel et de risque de règlement; b) les exigences limitant les grands risques; c) après l'entrée en vigueur de l'acte délégué visé à l'article 460, les exigences de liquidité relatives aux éléments entièrement quantifiables, uniformes et normalisés de risque de liquidité; d) les obligations de déclaration en ce qui concerne les points a), b) et c) et le levier; e) les obligations de publication. Le présent règlement ne régit pas les exigences de publication applicables aux autorités compétentes dans le domaine de la régulation et de la surveillance prudentielles 

In [38]:
# To dic
chunks_crr_dic = {i[:10] : i[10:] for i in chunks_crr}
chunks_crr_dic

{'Article pr': "emier Champ d'application Le présent règlement fixe des règles uniformes concernant les exigences prudentielles générales que tous les établissements faisant l'objet d'une surveillance en vertu de la directive 2013/36/UE respectent en ce qui concerne: a) les exigences de fonds propres relatives aux éléments entièrement quantifiables, uniformes et normalisés de risque de crédit, de risque de marché, de risque opérationnel et de risque de règlement; b) les exigences limitant les grands risques; c) après l'entrée en vigueur de l'acte délégué visé à l'article 460, les exigences de liquidité relatives aux éléments entièrement quantifiables, uniformes et normalisés de risque de liquidité; d) les obligations de déclaration en ce qui concerne les points a), b) et c) et le levier; e) les obligations de publication. Le présent règlement ne régit pas les exigences de publication applicables aux autorités compétentes dans le domaine de la régulation et de la surveillance prudentiel

### 3.2 - Statégie 2: "CHAPITRE", "SECTION", "Article" - Incomplete

In [39]:
def chunker_2(beginning: int, end:int, input_data_to_chunk: str) -> list:
    """Chunker by "CHAPITRE", "SECTION", "Article".
    ## Incomplete + non reliable
    "SECTION" peut etre espacé...
    Args:
        input_data_to_chunk: texte entrée a chunker apres conversion de pdf

    Returns:
        list: of chunks
    """
    article_splitter = RecursiveCharacterTextSplitter(
    separators= ["CHAPITRE", "SECTION", "Article"],
    chunk_size= 100,
    chunk_overlap=0)
    chunks= article_splitter.split_text(input_data_to_chunk)
    chunks = chunks[beginning:end]
    def ajouter_espace_article(texte): return re.sub(r'(Article\s+\d+)\s*([^\d\s])', r'\1  \2', texte)
    chunks=[ajouter_espace_article(i) for i in chunks]
    return chunks

#### Application a DSP2

In [40]:
chunks_v2 = chunker_2(1,120,dsp2_pdf_txt_fitz_clean)
chunks_v2

['Article premier Objet 1. La présente directive fixe les règles selon lesquelles les États membres distinguent les six catégories suivantes de prestataires de services de paiement:FR 23.12.2015 Journal officiel de l’Union européenne L 337/53 \n( 1 ) JO C 369 du 17.12.2011, p. 14. \n( 2 ) JO C 38 du 8.2.2014, p. 14.\n a) les établissements de crédit au sens de l’article 4, paragraphe 1, point 1), du règlement (UE) n o 575/2013 du Parlement européen et du Conseil ( 1 ), y compris leurs succursales au sens du point 17) dudit article 4, paragraphe 1, lorsque ces succursales sont situées dans l’Union, qu’il s’agisse de succursales d’établissements de crédit ayant leur siège dans l’Union ou, conformément à l’article 47 de la directive 2013/36/UE et au droit national, hors de l’Union; b) les établissements de monnaie électronique au sens de l’article 2, point 1), de la directive 2009/110/CE, y compris, conformément à l’article 8 de ladite directive et au droit national, une succursale d’un t

#### Application a AML 5

### 3.3 - Statégie 3: by "Article" / document + token 

In [41]:
import chromadb
from langchain_core.documents import Document

In [42]:
def chunker_doc(chunks_UE_dict: dict, Directive_source:str):
    """
    ## Option Facultative : dict -> list of docs + metadata
    
    Args:
        chunks_UE_dict (dict): Dictionnaire id sont articles et les clés contenus. 
        Directive_source (str): Nom ou référence de la directive UE. 

    Returns:
        dict: Document(id='1', metadata={'Directive_source': 'DPS2', 'N°article ': 'pre'}, page_content='mier Objet)
    """
    documents = []
    for article_key, article_content in chunks_UE_dict.items():
        doc = Document(
            page_content=article_content.lstrip(), 
            metadata={"Directive_source": Directive_source , "N°article ": article_key[7:].strip()}, 
            id=len(documents) + 1  # nombre de documents déjà créés 
        )
        documents.append(doc)
    return documents

In [43]:
chunks_dsp2_docs=chunker_doc(chunks_dsp2_v1_2,"DPS2")
chunks_dsp2_docs

[Document(id='1', metadata={'Directive_source': 'DPS2', 'N°article ': 'pre'}, page_content='mier Objet 1. La présente directive fixe les règles selon lesquelles les États membres distinguent les six catégories suivantes de prestataires de services de paiement:FR 23.12.2015 Journal officiel de l’Union européenne L 337/53 \n( 1 ) JO C 369 du 17.12.2011, p. 14. \n( 2 ) JO C 38 du 8.2.2014, p. 14.\n a) les établissements de crédit au sens de l’article 4, paragraphe 1, point 1), du règlement (UE) n o 575/2013 du Parlement européen et du Conseil ( 1 ), y compris leurs succursales au sens du point 17) dudit article 4, paragraphe 1, lorsque ces succursales sont situées dans l’Union, qu’il s’agisse de succursales d’établissements de crédit ayant leur siège dans l’Union ou, conformément à l’article 47 de la directive 2013/36/UE et au droit national, hors de l’Union; b) les établissements de monnaie électronique au sens de l’article 2, point 1), de la directive 2009/110/CE, y compris, conformémen

## 4 - Test chunking by retreival

### 4.1 - Test sur dsp2 

#### 4.1.1 Strat 1 - chunker par art:

##### Initialisation

In [44]:
from langchain_chroma import Chroma

def init_chromas_db_cosine(collection_name:str,embeddings ):
    global chromas_db_1 
    chromas_db_1= Chroma(
        collection_name=collection_name,
        embedding_function=embeddings,
        #persist_directory="/Users/oussa/Desktop/Github_perso/Advanced_RAG/vector_store",
        create_collection_if_not_exists=True,
        relevance_score_fn="cosine" # {"hnsw:space": "cosine"}
        )

In [45]:
init_chromas_db_cosine("dsp2", embeddings)

##### Input data in chromas

In [46]:
def input_chunks_chromasdb(chunks_dict:dict, nom_source:str, embedding_model, chemin:str, chunk_version:str):
    """Input chunks dans une base vectorielle ChromaDB à partir d'un dict.  

    Args:
        chunks_dict (dict): output of chunker_1_v1_step_2
        nom_source (str): exemple: nom de directive UE
        embedding_model (AzureOpenAI): 
        chemin (str): local enregistre

    Returns:
        Vector database: chroma_db
    """
    texts = list(chunks_dict.values())  # contenu des articles
    metadatas = [
        {"source": nom_source, "article": article_id}
        for article_id in chunks_dict.keys()
    ]
    global chroma_db_1_art
    chroma_db_1_art = Chroma.from_texts(
        texts=texts,
        metadatas=metadatas,
        persist_directory=f"{chemin}/Vector_store_{nom_source}_{chunk_version}",
        embedding=embedding_model,
        collection_metadata = {
        "hnsw:space": "cosine",          
        "hnsw:construction_ef": 200, # Nbr de voisins explorés lors de l'ajout
        "hnsw:M": 16   }             # Nbr de co par vecteur
    )
    return chroma_db_1_art

A faire : 

- Paramètres hnsw:construction_ef et hnsw:M : param influencent la qualité et la performance de l’index. 
- Pr trouver le meilleur compromis entre précision et efficacité en fonction de la taille et de la nature de vos données.

Attention: Immutabilité de certains paramètres : "hnsw:space", ne peuvent pas être modifiés après la création de la collection. il faudra recréer la collection

In [47]:
input_chunks_chromasdb(chunks_dsp2_v1_2,
                        "dsp2", 
                        embeddings,
                        "/Users/oussa/Desktop/Github_perso/",
                        "simple_by_art") 

In [48]:
print(chroma_db_1_art._collection.count()) 

351


##### Simple retreival test

In [49]:
question= "quel article renforce la protection des utilisateurs de services de paiement en imposant une obligation d'information claire et préalable sur les frais?"

In [50]:
results_1 = chroma_db_1_art.similarity_search_with_relevance_scores(question,4, filter={"source":"dsp2"})

In [51]:
def print_res(results:list):
    """Print output of similarity_search_with_relevance_scores

    Args:
        results (list of articles)
    """
    for doc, score in results:
        print(f"\n *** Simil={score:.3f}, {doc.metadata} : {doc.page_content} ")

In [52]:
print_res(results_1)


 *** Simil=0.614, {'article': 'Article 60', 'source': 'dsp2'} : Informations relatives aux frais supplémentaires ou aux réductions 1. Lorsque, aux fins de l’utilisation d’un instrument de paiement donné, le bénéficiaire applique des frais ou offre une réduction, il en informe le payeur avant l’initiation de l’opération de paiement. 2. Lorsque, aux fins de l’utilisation d’un instrument de paiement donné, le prestataire de services de paiement ou une autre partie intervenant dans l’opération applique des frais, il en informe l’utilisateur de services de paiement avant l’initiation de l’opération de paiement. 3. Le payeur n’est tenu d’acquitter les frais visés aux paragraphes 1 et 2 que s’il a eu connaissance de leur montant total avant l’initiation de l’opération de paiement TITRE IV DROITS ET OBLIGATIONS LIÉS À LA PRESTATION ET À L’UTILISATION DE SERVICES DE PAIEMENT CHAPITRE 1 Dispositions communes 

 *** Simil=0.614, {'article': 'Article 60', 'source': 'dsp2'} : Informations relative

#### 4.1.2 Strat 2 - chunks 1 doc pour 1 art : 

In [53]:
from langchain_core.documents import Document
from langchain_chroma import Chroma

La définition de collection_metadata a permis de booster les perfs passant de 0,5 a 0,61 (premier doc)

In [54]:
chroma_db_2_doc = Chroma( 
    collection_name="dsp2",
    embedding_function=embeddings,
    collection_metadata = {
        "hnsw:space": "cosine",          
        "hnsw:construction_ef": 200, # Nbr de voisins explorés lors de l'ajout
        "hnsw:M": 16}             # Nbr de co par vecteur
    )

In [55]:
chroma_db_2_doc.add_documents(chunks_dsp2_docs)
print(chroma_db_2_doc._collection.count()) 

117


In [56]:
results_doc=chroma_db_2_doc.similarity_search_with_relevance_scores(question,4)

In [57]:
for doc, score in results_doc:
    print(f"\n *** SIM={score:.3f}, {doc.metadata} : {doc.page_content} ")


 *** SIM=0.454, {'Directive_source': 'DPS2', 'N°article ': '60'} : Informations relatives aux frais supplémentaires ou aux réductions 1. Lorsque, aux fins de l’utilisation d’un instrument de paiement donné, le bénéficiaire applique des frais ou offre une réduction, il en informe le payeur avant l’initiation de l’opération de paiement. 2. Lorsque, aux fins de l’utilisation d’un instrument de paiement donné, le prestataire de services de paiement ou une autre partie intervenant dans l’opération applique des frais, il en informe l’utilisateur de services de paiement avant l’initiation de l’opération de paiement. 3. Le payeur n’est tenu d’acquitter les frais visés aux paragraphes 1 et 2 que s’il a eu connaissance de leur montant total avant l’initiation de l’opération de paiement TITRE IV DROITS ET OBLIGATIONS LIÉS À LA PRESTATION ET À L’UTILISATION DE SERVICES DE PAIEMENT CHAPITRE 1 Dispositions communes 

 *** SIM=0.452, {'Directive_source': 'DPS2', 'N°article ': '62'} : Frais applicabl

#### 4.1.3 Strat 3: Double chunk (par article pr 1 doc + chunk intérieur)

In [58]:
try :
    chroma_db_3_art_doc.delete_collection()
    print("Vector database supprimé")
except: print("Vector database inexistant")

Vector database inexistant


In [59]:
chroma_db_3_art_doc = Chroma( 
    collection_name="dsp2",
    embedding_function=embeddings,
    collection_metadata = {
        "hnsw:space": "cosine",          
        "hnsw:construction_ef": 200, # Nbr de voisins explorés lors de l'ajout
        "hnsw:M": 16}             # Nbr de co par vecteur
    )

In [60]:
chroma_db_3_art_doc._collection_name

'dsp2'

In [61]:
# 700 -> SIM=0.614
# 900 -> SIM=0.618

In [62]:
text_splitter_recursive_carac = RecursiveCharacterTextSplitter(
    chunk_size=900 ,
    chunk_overlap=90,
    # Ordre des séparateurs, du plus large au plus fin
    #separators=["\n\n", "\n", "."]
)

In [63]:
def chunker_3_all(spliter, documents:list):
    """
    - Chunk le contenu de chaque doc (1 article)
    - En entrée de la liste `documents` en plusieurs sous-documents (chunks).

    Args:
        documents (list): liste `documents`

    Returns:
        list: chunks
    """
    
    chunked_docs = []
    for doc in documents:
        # Récupérer le texte d'origine
        original_text = doc.page_content

        # Découper en plusieurs segments (chunks)
        chunks = spliter.split_text(original_text)

        # Construire la liste des nouveaux Documents en conservant les metadata
        for chunk in chunks:
            new_doc = Document(
                page_content=chunk,
                metadata=doc.metadata,  # On recopie les métadonnées du Document source
                id=len(chunked_docs) + 1
            )
            chunked_docs.append(new_doc)

    return chunked_docs

In [64]:
chunks_dsp2_docs

[Document(id='1', metadata={'Directive_source': 'DPS2', 'N°article ': 'pre'}, page_content='mier Objet 1. La présente directive fixe les règles selon lesquelles les États membres distinguent les six catégories suivantes de prestataires de services de paiement:FR 23.12.2015 Journal officiel de l’Union européenne L 337/53 \n( 1 ) JO C 369 du 17.12.2011, p. 14. \n( 2 ) JO C 38 du 8.2.2014, p. 14.\n a) les établissements de crédit au sens de l’article 4, paragraphe 1, point 1), du règlement (UE) n o 575/2013 du Parlement européen et du Conseil ( 1 ), y compris leurs succursales au sens du point 17) dudit article 4, paragraphe 1, lorsque ces succursales sont situées dans l’Union, qu’il s’agisse de succursales d’établissements de crédit ayant leur siège dans l’Union ou, conformément à l’article 47 de la directive 2013/36/UE et au droit national, hors de l’Union; b) les établissements de monnaie électronique au sens de l’article 2, point 1), de la directive 2009/110/CE, y compris, conformémen

In [65]:
final_chunked_docs= chunker_3_all(text_splitter_recursive_carac,chunks_dsp2_docs)
type(final_chunked_docs)

list

In [66]:
final_chunked_docs

[Document(id='1', metadata={'Directive_source': 'DPS2', 'N°article ': 'pre'}, page_content='mier Objet 1. La présente directive fixe les règles selon lesquelles les États membres distinguent les six catégories suivantes de prestataires de services de paiement:FR 23.12.2015 Journal officiel de l’Union européenne L 337/53 \n( 1 ) JO C 369 du 17.12.2011, p. 14. \n( 2 ) JO C 38 du 8.2.2014, p. 14.'),
 Document(id='2', metadata={'Directive_source': 'DPS2', 'N°article ': 'pre'}, page_content='a) les établissements de crédit au sens de l’article 4, paragraphe 1, point 1), du règlement (UE) n o 575/2013 du Parlement européen et du Conseil ( 1 ), y compris leurs succursales au sens du point 17) dudit article 4, paragraphe 1, lorsque ces succursales sont situées dans l’Union, qu’il s’agisse de succursales d’établissements de crédit ayant leur siège dans l’Union ou, conformément à l’article 47 de la directive 2013/36/UE et au droit national, hors de l’Union; b) les établissements de monnaie élect

In [67]:
chroma_db_3_art_doc.add_documents(final_chunked_docs)

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '13

In [68]:
resu_v2=chroma_db_3_art_doc.similarity_search_with_relevance_scores(question,4)

In [69]:
for doc, score in resu_v2:
    print(f"\n *** SIM={score:.3f}, {doc.metadata}, chunk n°{doc.id}: {doc.page_content} ")


 *** SIM=0.454, {'Directive_source': 'DPS2', 'N°article ': '60'}, chunk n°190: Informations relatives aux frais supplémentaires ou aux réductions 1. Lorsque, aux fins de l’utilisation d’un instrument de paiement donné, le bénéficiaire applique des frais ou offre une réduction, il en informe le payeur avant l’initiation de l’opération de paiement. 2. Lorsque, aux fins de l’utilisation d’un instrument de paiement donné, le prestataire de services de paiement ou une autre partie intervenant dans l’opération applique des frais, il en informe l’utilisateur de services de paiement avant l’initiation de l’opération de paiement. 3. Le payeur n’est tenu d’acquitter les frais visés aux paragraphes 1 et 2 que s’il a eu connaissance de leur montant total avant l’initiation de l’opération de paiement TITRE IV DROITS ET OBLIGATIONS LIÉS À LA PRESTATION ET À L’UTILISATION DE SERVICES DE PAIEMENT CHAPITRE 1 Dispositions communes 

 *** SIM=0.453, {'Directive_source': 'DPS2', 'N°article ': '62'}, chun

#### Semantic similarity

In [70]:
from langchain_experimental.text_splitter import SemanticChunker

Plusieurs méthodes sont disponibles pour effectuer un découpage basé sur la similarité sémantique, notamment percentile, standard deviation, interquartile, et gradient. Chaque méthode a ses avantages et est adaptée à des situations spécifiques

Cas d'utilisation : 

- Percentile: Approprié pour des documents avec des variations modérées dans la similarité sémantique. 
- Standard Deviation: Idéal pour les textes complexes ou spécialisés (ex. médical, juridique) où la structure est variée. 
- Interquartile: Convient aux documents avec une structure modérée, comme les historiques ou les articles scientifiques.
- Gradient: Utile pour des domaines spécifiques où les données sont fortement corrélées (ex. médical, légal)

In [71]:
text_splitter_semantic_v1 = SemanticChunker(embeddings)

The default way to split is based on percentile. In this method, all differences between sentences are calculated, and then any difference greater than the X percentile is split. The default value for X is 95:

In [72]:
text_splitter_semantic_v2_prc = SemanticChunker(embeddings,
                                            breakpoint_threshold_type="percentile",
                                            breakpoint_threshold_amount=60)# default is 95+ de segments.

The idea here is to apply anomaly detection on gradient array so that the distribution become wider and easy to identify boundaries in highly semantic data

In [73]:
text_splitter_semantic_v3_gdt = SemanticChunker(embeddings,
                                            breakpoint_threshold_type="gradient",
                                            breakpoint_threshold_amount=60) # default is 95 . Diminue + de segments.

In [74]:
docs_1 = text_splitter_semantic_v1.create_documents([chunks_dsp2_docs[2].page_content])

In [75]:
def source_exists_in_chroma_v3(chemin:str, source_name:str, 
                               embeddings, emb_model_name:str,
                               num_chunk_strat:int):
     """
     Vérifie si une source donnée existe déjà dans la base Chroma persistée.
     Args:
         chemin (str): Dossier de persistance de Chroma.
         source_name (str): Nom de la source à chercher dans les métadonnées.
         embedding_model: Modèle d'embedding utilisé pour initier Chroma.
 
     Returns: True si la source est déjà indexée, False sinon.
     """
     
     # Charge la base Chroma existante
     chroma_db = Chroma(
         persist_directory=chemin,
         embedding_function=embeddings)
 
     retriever = chroma_db.as_retriever()
     docs = retriever.invoke("objectif de la directive ?", filter={
        "$and": [ {"source": source_name}, 
                 {"Embeding_model": emb_model_name},
                 {"Chunk_strat": num_chunk_strat} ]
    })
 
     return len(docs) > 0

In [76]:
source_exists_in_chroma_v3(
    "/Users/oussa/Desktop/Github_perso/Advanced_RAG/vector_store/chromasdb",
    "aml_5",
    embeddings,
    'text-embedding-3-large',
    1)

False

In [77]:
emb_model_name = embeddings.model_dump()["model"]
emb_model_name

'text-embedding-3-large_standard'

In [78]:
chroma_db = Chroma(
    persist_directory="/Users/oussa/Desktop/Github_perso/Advanced_RAG/vector_store/chromasdb",
         embedding_function=embeddings)


In [79]:
retriever = chroma_db.as_retriever()

In [80]:
docs = retriever.invoke("objectif de la directive ?", filter={"source": "aml_5"})

In [81]:
docs

[]

In [82]:
chroma_db._collection.count()

69

In [83]:
for i in range(len(docs_1)):
    print("Chunk:",i, docs_1[i].page_content)

Chunk: 0 Exclusions La présente directive ne s’applique pas: a) aux opérations de paiement effectuées exclusivement en espèces et allant directement du payeur au bénéficiaire, sans l’intervention du moindre intermédiaire; b) aux opérations de paiement allant du payeur au bénéficiaire, par l’intermédiaire d’un agent commercial habilité par contrat à négocier ou à conclure la vente ou l’achat de biens ou de services pour le compte du payeur uniquement ou du bénéficiaire uniquement; c) au transport physique de billets de banque et de pièces à titre professionnel, y compris leur collecte, leur traitement et leur remise; d) aux opérations de paiement consistant en la collecte et la remise d’espèces à titre non professionnel, dans le cadre d’une activité à but non lucratif ou caritative; e) aux services pour lesquels des espèces sont fournies par le bénéficiaire au bénéfice du payeur dans le cadre d’une opération de paiement, à la demande expresse de l’utilisateur de services de paiement for

In [84]:
docs_2 = text_splitter_semantic_v2_prc.create_documents([chunks_dsp2_docs[2].page_content])

In [85]:
for i in range(len(docs_2)):
    print("Chunk:",i, docs_2[i].page_content)

Chunk: 0 Exclusions La présente directive ne s’applique pas: a) aux opérations de paiement effectuées exclusivement en espèces et allant directement du payeur au bénéficiaire, sans l’intervention du moindre intermédiaire; b) aux opérations de paiement allant du payeur au bénéficiaire, par l’intermédiaire d’un agent commercial habilité par contrat à négocier ou à conclure la vente ou l’achat de biens ou de services pour le compte du payeur uniquement ou du bénéficiaire uniquement; c) au transport physique de billets de banque et de pièces à titre professionnel, y compris leur collecte, leur traitement et leur remise; d) aux opérations de paiement consistant en la collecte et la remise d’espèces à titre non professionnel, dans le cadre d’une activité à but non lucratif ou caritative; e) aux services pour lesquels des espèces sont fournies par le bénéficiaire au bénéfice du payeur dans le cadre d’une opération de paiement, à la demande expresse de l’utilisateur de services de paiement for

In [86]:
docs_3 = text_splitter_semantic_v3_gdt.create_documents([chunks_dsp2_docs[2].page_content])
print(docs_3[0].page_content)

Exclusions La présente directive ne s’applique pas: a) aux opérations de paiement effectuées exclusivement en espèces et allant directement du payeur au bénéficiaire, sans l’intervention du moindre intermédiaire; b) aux opérations de paiement allant du payeur au bénéficiaire, par l’intermédiaire d’un agent commercial habilité par contrat à négocier ou à conclure la vente ou l’achat de biens ou de services pour le compte du payeur uniquement ou du bénéficiaire uniquement; c) au transport physique de billets de banque et de pièces à titre professionnel, y compris leur collecte, leur traitement et leur remise; d) aux opérations de paiement consistant en la collecte et la remise d’espèces à titre non professionnel, dans le cadre d’une activité à but non lucratif ou caritative; e) aux services pour lesquels des espèces sont fournies par le bénéficiaire au bénéfice du payeur dans le cadre d’une opération de paiement, à la demande expresse de l’utilisateur de services de paiement formulée jus

In [87]:
for i in range(len(docs_3)):
    print("Chunk",i, ":",docs_3[i].page_content)

Chunk 0 : Exclusions La présente directive ne s’applique pas: a) aux opérations de paiement effectuées exclusivement en espèces et allant directement du payeur au bénéficiaire, sans l’intervention du moindre intermédiaire; b) aux opérations de paiement allant du payeur au bénéficiaire, par l’intermédiaire d’un agent commercial habilité par contrat à négocier ou à conclure la vente ou l’achat de biens ou de services pour le compte du payeur uniquement ou du bénéficiaire uniquement; c) au transport physique de billets de banque et de pièces à titre professionnel, y compris leur collecte, leur traitement et leur remise; d) aux opérations de paiement consistant en la collecte et la remise d’espèces à titre non professionnel, dans le cadre d’une activité à but non lucratif ou caritative; e) aux services pour lesquels des espèces sont fournies par le bénéficiaire au bénéfice du payeur dans le cadre d’une opération de paiement, à la demande expresse de l’utilisateur de services de paiement fo

Deux observations: 
- seul le gradient est capable de diviser les textes juridiques.
- quelque soit la valeur choisie ,les points de rupture détectés restent les mêmes =>  le texte est divisé de la meme facon 

Conclusions: 
- Les méthodes de segmentation basées sur des seuils statistiques globaux peuvent ne pas détecter efficacement ces transitions discrètes. 
- La méthode du gradient analyse les variations locales des distances cosines entre les phrases, ce qui la rend plus apte à identifier les points de rupture dans des textes où les changements sémantiques sont moins prononcés
- pas d effet des seuil car les textes sont relativement uniformes ou si les différences sont minimes

Résultat: on retient la méthode de gradient pour le semantic segmentation